In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from geojson import MultiLineString
from folium.plugins import HeatMap
from glob import glob
from folium import plugins
import branca.colormap

In [14]:
def get_coordinates(row):
    '''Method takes a string of coordinates from multiline column 
    and splits them into individual pair of coordinates consisting of latitudes and longitudes.
    @return list of coordinates
    '''
    temp_list = []
    coordinates = row.replace("(","").replace(")","").split(",")
    for j in coordinates:
        cord_list= []
        cord_list.append(float(j.split()[1]))
        cord_list.append(float(j.split()[0]))
        temp_list.append(cord_list)

            
    return temp_list



In [16]:
# reading speed limits
df_speed_all = pd.read_csv("Speed_Limits.csv")

# PARSING DATE to convert from object to date format 
df_speed_all['CREATED_DT'] = pd.to_datetime(df_speed_all['CREATED_DT'], format = '%Y-%m-%d')

# filtering data for year 2018
df_speed = df_speed_all[df_speed_all['CREATED_DT'].dt.year == 2018]

# remove own index with default index 
df_speed.reset_index(inplace = True, drop = True)

# fetching only speed and coordinates data from multiline column into new dataframe
# df_speed_loc has only speeds and location coordinates
df_speed_loc = df_speed[['SPEED','multiline']]

# removing extra characters from location
df_speed_loc['multiline'] = df_speed_loc['multiline'].apply(lambda x: x[16:])
            
#calling get_coordinates method to get individual coordinates from column multiline or each row
# storing the coordinates list returned from the method into new column coordinates
df_speed_loc['coordinates'] = df_speed_loc['multiline'].apply(get_coordinates)
    
df_speed_loc.head()


BOUND           object
CREATED_DT      object
DISTANCE       float64
SPEED            int64
STREET_NAME     object
multiline       object
dtype: object


C:\Users\nehas\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nehas\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,SPEED,multiline,coordinates
0,40,"((-114.034764352652 50.996224143932, -114.0348...","[[50.996224143932, -114.034764352652], [50.996..."
1,60,"((-114.068566084381 51.166713797578, -114.0685...","[[51.166713797578, -114.068566084381], [51.166..."
2,80,"((-113.918850808952 51.062448223076, -113.9186...","[[51.062448223076, -113.918850808952], [51.062..."
3,40,"((-113.975198107169 50.900558749208, -113.9750...","[[50.900558749208, -113.975198107169], [50.901..."
4,60,"((-114.048515198985 51.126022055968, -114.0487...","[[51.126022055968, -114.048515198985], [51.126..."


In [8]:
# defining dict of colors to be used to represent data on the map
dict_colors = {20: '#F5F216', 30: '#F5CB16', 35: '#FFD82F', 40: '#D7B520', 45: '#FFD733', 60: '#FFAF00', 70: '#FF8C00', 80: '#FF6900', 90: '#FF3400', 100: '#FF0000', 110: '#A54D1D'}



In [18]:
# Create the map for Calgary by defining Calgary coordinates
locat = [51.049999, -114.066666]
m_new = folium.Map(location=locat, zoom_start=10)


# looping through tuple of shape (rows, columns), 0 defines the index for rows fro dataframe df_speed_loc
# fetching the coordinates and plotting the same on the map 
# where a set of coordinates defines the beginning and end of line for given speed at the ith row in column 'SPEED'.
for i in range (df_speed_loc.shape[0]):
    coordinates = df_speed_loc.loc[i,'coordinates']
    my_PolyLine = folium.PolyLine(locations=coordinates, weight=3, color = dict_colors[df_speed_loc.loc[i,'SPEED']])
    m_new.add_child(my_PolyLine)

# Creates a ColorMap based on linear interpolation of a set of colors over a given index 
colormap = branca.colormap.linear.YlOrRd_09.scale(0, 100)

# Splits the LinearColormap into a StepColormap.
# index defines values corresponding to each colour bound
colormap = colormap.to_step(index=[20, 30, 35, 40, 45, 60, 70, 80, 90, 100, 110])
colormap.caption = 'Speed Limits'

# adding colormap plot of coordinates to the map
colormap.add_to(m_new)
m_new.save('Speed_Limit_2018.html')

m_new